Import And Connection

In [7]:
!pip install pypyodbc


Defaulting to user installation because normal site-packages is not writeable


In [11]:
import pyodbc as odbc
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [12]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = r'DESKTOP-L3GBMQ5\SQLEXPRESS'
DATABASE_NAME = 'Case_Studies'

connection_string = (
    f"DRIVER={{{DRIVER_NAME}}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)

conn = odbc.connect(connection_string)
print("Connected successfully!")

Connected successfully!


-------------------

Regional Performance

In [32]:
sql_region = """
SELECT 
    Region
    ,SUM(total_sales) as Total_Sales
    ,SUM(total_sales * 0.25) AS Total_Profit
FROM
    retail_sales_cleaned_3
GROUP BY
    Region
ORDER BY
    Total_Sales DESC;
"""

In [33]:
df_region = pd.read_sql(sql_region, conn)
df_region

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_7812\2719363155.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_region = pd.read_sql(sql_region, conn)


,Region,Total_Sales,Total_Profit
0,North,619477.729387,154869.432347
1,East,572738.510361,143184.627590
2,West,538043.409042,134510.852261
3,South,525178.162788,131294.540697


State-Lever Performance

In [37]:
df = pd.read_sql("SELECT * FROM [dbo].[retail_sales_cleaned_3];", conn)
df.head()


C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_7812\2033534674.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM [dbo].[retail_sales_cleaned_3];", conn)


,order_id,order_date,customer_id,customer_name,region,product_category,product_name,quantity,unit_price,discount,total_sales,payment_method,delivery_status,order_year,order_month,segment,state
0,O-1000,2024-01-01,C-208,Jane Smith,South,Furniture,T-Shirt,10,805.520020,00:01:00,7974.647949,Credit Card,Pending,2024.0,1.0,Medium,Texas
1,O-1001,2024-01-02,C-365,Ali Hassan,North,Furnitre,Sofa,3,1550.010010,00:25:00,3487.522461,Cash,Cancelled,2024.0,1.0,Low,Unknown
2,O-1002,2024-01-03,C-498,Jane Smith,West,Furniture,Laptop,1,1187.380005,00:10:00,1068.641968,Bank Transfer,Pending,2024.0,1.0,Low,California
3,O-1003,2024-01-04,C-371,Ali Hassan,East,Electronics,Headphones,2,932.390015,00:07:00,1734.245361,Cash,Cancelled,2024.0,1.0,Low,New York
4,O-1004,2024-01-05,C-218,Jane Smith,West,Clothing,Sofa,10,279.369995,00:17:00,2318.770996,Cash,Returned,2024.0,1.0,Low,California


In [46]:
sql_state = """
SELECT
    region
    ,state
    ,SUM(total_sales) AS Total_Sales
    ,SUM(total_sales * 0.25) AS Total_Profit
FROM
    retail_sales_cleaned_3
GROUP BY
    region
    ,state
ORDER BY 
    Total_Sales DESC;
"""

In [47]:
df_state = pd.read_sql(sql_state, conn)
df_state

C:\Users\GIGABYTE\AppData\Local\Temp\ipykernel_7812\2013090188.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_state = pd.read_sql(sql_state, conn)


,region,state,Total_Sales,Total_Profit
0,North,Unknown,619477.729387,154869.432347
1,East,New York,572738.510361,143184.627590
2,West,California,538043.409042,134510.852261
3,South,Texas,525178.162788,131294.540697


Ranking & Identifying Loss-Making States

In [48]:
# Sort by Profit
df_state_sorted = df_state.sort_values(by='Total_Profit', ascending=False)


In [49]:
# Top 5 Performance
top_5_states = df_state_sorted.head(5)

In [50]:
# Bottom 5 Performance
bottom_5_states = df_state_sorted.tail(5)

In [51]:
# Loss-Making States
loss_making_states = df_state_sorted[df_state_sorted['Total_Profit'] < 0]

In [52]:
display(top_5_states, bottom_5_states, loss_making_states)

,region,state,Total_Sales,Total_Profit
0,North,Unknown,619477.729387,154869.432347
1,East,New York,572738.510361,143184.627590
2,West,California,538043.409042,134510.852261
3,South,Texas,525178.162788,131294.540697


,region,state,Total_Sales,Total_Profit
0,North,Unknown,619477.729387,154869.432347
1,East,New York,572738.510361,143184.627590
2,West,California,538043.409042,134510.852261
3,South,Texas,525178.162788,131294.540697


,region,state,Total_Sales,Total_Profit


Regional Contribution & Profit Margin

In [53]:
# Contribution %
df_region["Sales_Contribution_%"] = (df_region["Total_Sales"] / df_region["Total_Sales"].sum()) * 100

In [54]:
# Profit Margin
df_region["Profit_Margin_%"] = (df_region["Total_Profit"] / df_region["Total_Sales"]) * 100

In [55]:
df_region

,Region,Total_Sales,Total_Profit,Sales_Contribution_%,Profit_Margin_%
0,North,619477.729387,154869.432347,27.465964,25.0
1,East,572738.510361,143184.627590,25.393673,25.0
2,West,538043.409042,134510.852261,23.855387,25.0
3,South,525178.162788,131294.540697,23.284976,25.0


In [56]:
df_state.to_csv("state_performance.csv", index=False)
df_region.to_csv("region_performance.csv", index=False)